In [ ]:
import numpy
from scipy.sparse import csc_matrix, csr_matrix, hstack, identity
from scipy.sparse.linalg import inv, norm

import pyibm

from helper import *

In [ ]:
%matplotlib qt

In [ ]:
pyibm.__version__

In [ ]:
xlim, ylim = (-15.0, 15.0), (-15.0, 15.0)
xbox, ybox = (-0.75, 0.75), (-0.75, 0.75)
dx, dy = 0.05, 0.05  # grid spacings in the uniform region

config = {'x': {'start': xlim[0],
                'segments': [{'end': xbox[0],
                              'width': dx,
                              'stretching': 1.03,
                              'reverse': True},
                             {'end': xbox[1],
                              'width': dx,
                              'stretching': 1.0},
                             {'end': xlim[1],
                              'width': dx,
                              'stretching': 1.01}]},
          'y': {'start': ylim[0],
                'segments': [{'end': ybox[0],
                              'width': dy,
                              'stretching': 1.04,
                              'reverse': True},
                             {'end': ybox[1],
                              'width': dy,
                              'stretching': 1.0},
                             {'end': ylim[1],
                              'width': dy,
                              'stretching': 1.04}]}}

grid = pyibm.GridBase(config=config)
gridc = pyibm.GridCellCentered(grid=grid)
gridx = pyibm.GridFaceX(grid=grid)
gridy = pyibm.GridFaceY(grid=grid)

In [ ]:
print(gridc)
print(gridx)
print(gridy)

In [ ]:
GHat = pyibm.assemble_GHat(gridc, gridx, gridy)
DHat = pyibm.assemble_DHat(gridc, gridx, gridy)

print('GHat:')
print_matrix_info(GHat)
print('DHat:')
print_matrix_info(DHat)

In [ ]:
Re = 100.0
LHat = 1 / Re * pyibm.assemble_LHat(gridx, gridy)
print_matrix_info(LHat)

In [ ]:
dt = 0.01
BN = pyibm.assemble_BN(gridx, gridy, dt=dt, alpha=0.5, N=3, L=LHat)
print_matrix_info(BN)

In [ ]:
DBNG = DHat @ BN @ GHat
print_matrix_info(DBNG)
is_symmetric(DBNG)

In [ ]:
alpha = 0.5
size = gridx.size + gridy.size
I = identity(size)
A = 1 / dt * I - alpha * LHat
print_matrix_info(A)
is_symmetric(A)

In [ ]:
N_min, N_max = 1, 3
for N in range(N_min, N_max + 1):
    print('Truncation order: ', N)
    BN = pyibm.assemble_BN(gridx, gridy, dt=dt, alpha=0.5, N=N, L=LHat)
    DBNG = DHat @ BN @ GHat
    print_matrix_info(DBNG)
    print('Condition number: ', condition_number(DBNG))
    print('Is symmetric? ', is_symmetric(DBNG))
    print('Min abs data: ', numpy.min(abs(DBNG).data))
    M = A @ BN - I
    print('L2(A BN - I) = ', norm(M))
#     plot_matrix(DBNG, cmap='viridis', markersize=10)